In [39]:
import numpy as np
import torch

In [40]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [41]:
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [42]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [43]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [44]:
def model(x,w,b):
    return x @ w.t() + b

In [45]:
def gd(weight,bias,inputs,targets,epochs=1000):
    for i in range(epochs):
        preds = model(inputs,weight,bias)
        loss = mse(preds, targets)
        loss.backward()
        with torch.no_grad():
            weight -= w.grad * 1e-5
            bias -= b.grad * 1e-5
            weight.grad.zero_()
            bias.grad.zero_()
    return preds,loss

In [46]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.6703, -0.0974, -0.0182],
        [ 0.2507,  0.9366,  1.5468]], requires_grad=True)
tensor([-1.7200, -0.2902], requires_grad=True)


In [47]:
preds,loss = gd(w,b,inputs,targets,epochs=1700)
print(loss)
print(preds)
print(targets)

tensor(1.3949, grad_fn=<DivBackward0>)
tensor([[ 56.9669,  70.1916],
        [ 81.6304, 101.2677],
        [120.3175, 131.7873],
        [ 21.5614,  36.6915],
        [100.2973, 120.3074]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [48]:
import torch.nn as nn

In [49]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [50]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [51]:
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [52]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.]])
tensor([[ 81., 101.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 56.,  70.],
        [119., 133.]])


In [53]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3055, -0.2505,  0.4709],
        [-0.3795, -0.4975,  0.3869]], requires_grad=True)
Parameter containing:
tensor([ 0.4789, -0.3132], requires_grad=True)


In [54]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3055, -0.2505,  0.4709],
         [-0.3795, -0.4975,  0.3869]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4789, -0.3132], requires_grad=True)]

In [62]:
preds = model(inputs)
preds

tensor([[-18.3576, -44.7093],
        [-19.2286, -53.8624],
        [-32.3570, -77.5493],
        [-24.0279, -46.0958],
        [-11.6877, -47.1727],
        [-18.3576, -44.7093],
        [-19.2286, -53.8624],
        [-32.3570, -77.5493],
        [-24.0279, -46.0958],
        [-11.6877, -47.1727],
        [-18.3576, -44.7093],
        [-19.2286, -53.8624],
        [-32.3570, -77.5493],
        [-24.0279, -46.0958],
        [-11.6877, -47.1727]], grad_fn=<AddmmBackward>)

In [63]:
import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(16974.5488, grad_fn=<MseLossBackward>)


In [57]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [64]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if epoch+1 == num_epochs:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [65]:
fit(200, model, loss_fn, opt, train_dl)

Epoch [200/200], Loss: 0.9829


In [66]:
preds = model(inputs)
preds

tensor([[ 57.3204,  70.3854],
        [ 82.9779, 100.8081],
        [116.8385, 132.4710],
        [ 21.4682,  37.8083],
        [103.1199, 118.8576],
        [ 57.3204,  70.3854],
        [ 82.9779, 100.8081],
        [116.8385, 132.4710],
        [ 21.4682,  37.8083],
        [103.1199, 118.8576],
        [ 57.3204,  70.3854],
        [ 82.9779, 100.8081],
        [116.8385, 132.4710],
        [ 21.4682,  37.8083],
        [103.1199, 118.8576]], grad_fn=<AddmmBackward>)

In [67]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])